In [1]:
!pip install -q tensorflow==2.15.0

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Define paths
train_dir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [6]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)  

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(29, activation='softmax')  
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [12]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/10
2447/2447 [==============================] - 547s 223ms/step - loss: 0.9530 - accuracy: 0.7010 - val_loss: 0.5523 - val_accuracy: 0.8164
Epoch 2/10
2447/2447 [==============================] - 253s 104ms/step - loss: 0.1694 - accuracy: 0.9429 - val_loss: 0.6258 - val_accuracy: 0.8360
Epoch 3/10
2447/2447 [==============================] - 240s 98ms/step - loss: 0.0936 - accuracy: 0.9684 - val_loss: 0.5745 - val_accuracy: 0.8723
Epoch 4/10
2447/2447 [==============================] - 243s 99ms/step - loss: 0.0649 - accuracy: 0.9782 - val_loss: 0.5994 - val_accuracy: 0.8703
Epoch 5/10
2447/2447 [==============================] - 243s 99ms/step - loss: 0.0516 - accuracy: 0.9831 - val_loss: 0.5073 - val_accuracy: 0.8951
Epoch 6/10
2447/2447 [==============================] - 233s 95ms/step - loss: 0.0431 - accuracy: 0.9862 - val_loss: 0.4970 - val_accuracy: 0.8969
Epoch 7/10
2447/2447 [==============================] - 229s 93ms/step - loss: 0.0364 - accuracy: 0.9889 - val_loss:

In [13]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

test_dir = "/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test"
test_images = sorted(os.listdir(test_dir))  

correct = 0
total = 0

for filename in test_images:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=(64, 64))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  

    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    true_class_label = filename[0]

    label_map = train_generator.class_indices
    label_list = list(label_map.keys())
    predicted_label = label_list[predicted_class]

    if predicted_label == true_class_label:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Manual Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 0s 16ms/step
Manual Test Accuracy: 0.9286


In [14]:
model.save('asl_classifier.h5',save_format="h5")

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from IPython.display import FileLink
FileLink("asl_classifier.h5")

/kaggle/working/asl_classifier.h5